Compute some values from Spinorama following Olive&al

In [5]:
import math
import pandas as pd
import altair as alt
import numpy as np
from src.spinorama.load import parse_all_speakers, parse_graphs_speaker
from src.spinorama.graph import graph_params_default

# df = parse_graphs_speaker('Adam', 'Adam S2V', 'klippel')
df = parse_graphs_speaker('Neumann', 'Neumann KH 80', 'klippel')
# df = parse_graphs_speaker('Genelec', 'Genelec 8030A', 'princeton')
# print(df)

In [81]:
def octave(N):
    """compute 1/N octave band"""
    p = pow(2,1/N)
    p_band= pow(2,1/(2*N))
    iter = int((N*10+1)/2)
    center = [1000 / p**i for i in range(iter,0,-1)]+[1000*p**i for i in range(0,iter,1)]
    return [(c/p_band,c*p_band) for c in center]

octave20 = octave(20)




In [116]:
def aad(dfu):
    # mean betwenn 200hz and 400hz
    y_ref = np.mean(dfu.loc[(dfu.Freq>=200) & (dfu.Freq<=400)].dB)
    #print(y_ref)
    sum = 0
    n = 0
    # 1/20 octave
    for (omin, omax) in octave(20):
        # 100hz to 16k hz
        if omin < 100:
            continue
        if omax > 16000:
            break
        sum += abs(y_ref-np.mean(dfu.loc[(dfu.Freq>=omin) & (dfu.Freq<omax)].dB))
        n += 1
    return sum/n

def nbd(dfu):
    sum = 0
    n = 0
    # 1/2 octave
    for (omin, omax) in octave(2):
        # 100hz to 12k hz
        if omin < 100:
            continue
        if omax > 12000:
            break
        y = dfu.loc[(dfu.Freq>=omin) & (dfu.Freq<omax)].dB
        y_avg = np.mean(y)
        # don't sample, take all points in this octave
        sum += np.mean(np.abs(y_avg-y))
        n += 1
    return sum/n

def lfx(dfu):
    y_ref = np.mean(dfu.loc[(dfu.Freq>=300) & (dfu.Freq<=10000)].dB)-6
    # find first freq such that y[freq]<y_ref-6dB
    y = dfu.loc[(dfu.Freq<300)&(dfu.dB<y_ref)].Freq.max()
    return y

    
df_onaxis = df['CEA2034'].loc[lambda df: df.Measurements == 'On Axis']
df_lw = df['CEA2034'].loc[lambda df: df.Measurements == 'Listening Window']

print('aad={0}'.format(aad(df_onaxis)))
print('nbd={0}'.format(nbd(df_onaxis)))
print('lfx={0}'.format(lfx(df_lw)))

aad=0.6374662068965525
nbd=0.32108885482643135
lfx=52.7344


In [96]:
t = dfu.loc[(dfu.Freq>=200) & (dfu.Freq<=400)].dB
m = np.mean(t)
np.mean(np.abs(t-m))


0.20390000000000086

In [109]:
dfu.loc[(dfu.Freq<300) & (dfu.dB<70)]

,Freq,Measurements,dB
200,20.5078,On Axis,55.7961
201,21.2402,On Axis,51.9241
202,21.9727,On Axis,53.8309
203,22.7051,On Axis,55.0830
204,23.4375,On Axis,41.6382
205,24.1699,On Axis,43.5598
206,25.6348,On Axis,51.0286
207,26.3672,On Axis,53.8528
208,27.0996,On Axis,56.2727
209,27.8320,On Axis,58.5374
